In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# =============================================================================
# Paramètres du problème
# =============================================================================
L     = 0.30         # Épaisseur du mur [m]
Nx    = 100         # Nombre d'intervalles spatiaux => Nx+1 nœuds
dx    = L / Nx       # Pas d'espace
x     = np.linspace(0, L, Nx+1)

k     = 1        # Conductivité thermique [W/(m*K)]
h     = 1         # Coefficient de convection [W/(m^2*K)]
rho   = 2000.0       # Masse volumique [kg/m^3]
C_v   = 1000.0       # Capacité thermique massique [J/(kg*K)]
q     = 2000.0       # Taux volumique d’émission de chaleur [W/m^3]
dL    = 0.05         # Longueur caractéristique pour moduler la source
T_a   = -10.0        # Température ambiante extérieure [°C]
T_i   = 20.0         # Température imposée à x=L [°C]

dt     = 21   # Pas de temps [s]
t_final= 100000      # Durée totale de simulation [s]
nSteps = int(t_final/dt)

# Condition convective (de Robin) à x=0 (face externe du mur): -k*dT/dx=h(Ta-T)
c1 = -k
c2 = h
c3 = -h*T_a

# Condition de Neumann à x=L (face interne du mur): dT/dx=0 - flux net de chaleur est 0
d1 = 0
d2 = 1
d3 = -T_i

xi     = 1        # xi=0.5 => Crank–Nicolson ; xi=1 => implicite total.

# =============================================================================
# Condition initiale
# =============================================================================
T_w = ( (k/L)*T_i + h*T_a ) / ( (k/L) + h )
T   = T_w + (T_i - T_w)*(x/L)


# =============================================================================
# Construction du Laplacien "classique" 1D
# =============================================================================
Lap = np.zeros((Nx+1, Nx+1))
for i in range(1, Nx):
    Lap[i, i-1] =  1.0
    Lap[i, i]   = -2.0
    Lap[i, i+1] =  1.0
# Les conditions aux limites seront ajustées dans la boucle en temps.

# Matrices
A = np.diag(-2*np.ones(int(N+1)),0) + np.diag(np.ones(int(N)),-1) + np.diag(np.ones(int(N)),1)

A[0,0] = 2 * c2 * dx - 3 * c1
A[0,1] = 4 * c1
A[0,2] = -c1
A[int(Nx),int(Nx)] = 3 * d1 + 2 * d2 * dx
A[int(Nx),int(Nx-1)] = -4 * d1 
A[int(Nx),int(Nx-2)] = d1

M = np.eye(Nx)  # Create an identity matrix of size n
M[0, :] = 0  # Set the first row to zeros
M[-1, :] = 0  #

S = q*np.exp(-((x-L)/dL)**2) # Pourquoi c'est pas la même formule que dans le procédurier?
b = -S / k * dx ** 2
b[0] = -2 * c3 * dx
b[int(Nx)] = -2 * d3 * dx


# =============================================================================
# Boucle en temps
# =============================================================================

for n in range(nSteps):
    # Coeffs utiles
    coeffM  = (C_v * rho) / dt
    alpha = (C_v * rho) / k
    coeff_alp = (dt/(alpha*dx*dx))
    coeffCN = (k / (dx*dx)) * dt #coeffCN = (dt/ (alpha*(dx*dx))) 

    # A_prime = M - xi*dt*k*Lap / dx^2
    A_prime = coeffM * np.eye(Nx+1) - ( xi * coeffCN ) * Lap
    A_prime_alt = M - coeff_alp* xi * A
    
    # b_vec = M*T^n + (1-xi)*dt*k*Lap/dx^2 * T^n  +  S(x)*dt
    b_vec   = coeffM * T
    b_vec  += ( (1.0 - xi)* coeffCN ) * (Lap @ T)
    
    # Terme source
    Sx = q * (1 + ((x - L)/dL)**2)
    b_vec += Sx * dt
    
    # Condition de convection (Robin) à x=0
    A_prime[0, :] = 0.0
    A_prime[0, 0] = coeffM + xi * coeffCN * (k/dx + h) / k
    A_prime[0, 1] = - xi * coeffCN * (k/dx) / k
    b_vec[0]      = ( coeffM*T[0]
                      + (1.0 - xi)* coeffCN*(Lap[0,:] @ T)
                      + (Sx[0]*dt)
                      + xi * coeffCN * (h*T_a)/k )
    
    # Condition de Dirichlet T(L) = T_i
    A_prime[Nx, :]  = 0.0
    A_prime[Nx, Nx] = 1.0
    b_vec[Nx]       = T_i
    
    # Résolution
    T_new = np.linalg.solve(A_prime, b_vec)
    

    print("max", n*dt,"s" ,np.max(T_new))
    # Mise à jour
    T = T_new

# =============================================================================
# Affichage
# =============================================================================
plt.figure(figsize=(8,5))
plt.plot(x, T, 'o-', label=f'T(x) à t={t_final:.1f}s (xi={xi})')
plt.xlabel('x [m]')
plt.ylabel('Température [°C]')
plt.title('Distribution de Température dans le mur')
plt.legend()
plt.grid(True)
plt.show()

max 0 s 27.764501510467188
max 21 s 41.55910606323346
max 42 s 54.78792218674357
max 63 s 67.55615128805195
max 84 s 79.9323600668547
max 105 s 91.96625470099622
max 126 s 103.69265838739926
max 147 s 115.14074317221305
max 168 s 126.33476342527231
max 189 s 137.29240511136666
max 210 s 148.03075430333243
max 231 s 158.56509198813177
max 252 s 168.90600271297376
max 273 s 179.06437128678218
max 294 s 189.05242294143252
max 315 s 198.8764560515251
max 336 s 208.54428054784873
max 357 s 218.06389717210757
max 378 s 227.4426628067283
max 399 s 236.68489615954593
max 420 s 245.79609198483956
max 441 s 254.78216454678616
max 462 s 263.64865422358656
max 483 s 272.3982865355545
max 504 s 281.0351282638438
max 525 s 289.5629767059369
max 546 s 297.98769488349365
max 567 s 306.31045287633185
max 588 s 314.53424805504807
max 609 s 322.66202080523146
max 630 s 330.6968915584928
max 651 s 338.6431405480499
max 672 s 346.50123410600156
max 693 s 354.273507041733
max 714 s 361.96217093961843
max 73

KeyboardInterrupt: 